In [ ]:
#se realizara un analisis de ventas, son archivos csv que se encuentran separado un archivo por mes

In [ ]:
import pandas as pd
import os 
import matplotlib.pyplot as plt

### Tarea 1: combinar los 12 meses de datos de ventas en un solo csv y luego leer el nuevo archivo

In [ ]:
path= "C:/Users/milag/proyectos-python/Analisis-Ventas/Ventas-data"
files= os.listdir(path)
DF= pd.DataFrame()
for file in files :
    df= pd.read_csv(path+"/"+file)
    DF=pd.concat([df,DF])
DF.to_csv(r'C:\Users\milag\proyectos-python\Analisis-Ventas\ventas_12meses.csv',index=False)


In [ ]:
#leer el nuevo archivo
data_completa= pd.read_csv(r'C:\Users\milag\proyectos-python\Analisis-Ventas\ventas_12meses.csv')

In [ ]:
data_completa.head()

### Tarea2: limpieza y transformación de la data 

In [ ]:
#verificar cuanta data de las columnas del df estan vacias
data_completa.isnull().sum()

In [ ]:
#eliminar las filas vacias 
#0 filas
data_completa.dropna(axis=0, how='any',inplace=True)

In [ ]:
#verificar cuanta data de las columnas del df estan vacias
data_completa.isnull().sum()

In [ ]:
#cambiar el tipo de dato
#primero verificar el continido 
data_completa['Quantity Ordered'].value_counts()

In [ ]:
#eliminar las filas que tienen por dato los nombres de la columna
# se obtienes los indices data_completa[data_completa['Quantity Ordered']== 'Quantity Ordered'].index
data_completa.drop(data_completa[data_completa['Quantity Ordered']== 'Quantity Ordered'].index,axis=0, inplace=True)

In [ ]:
#NOTA: se tiene la data del 0 al 2  data_completa.iloc[:3]

In [ ]:
#verificando que la data se haya eliminado
data_completa[data_completa['Quantity Ordered']== 'Quantity Ordered']

In [ ]:
#verificar que tipo de datos tienen las columnas
data_completa.dtypes

In [ ]:
#cambio de tipo
data_completa['Quantity Ordered']= data_completa['Quantity Ordered'].astype(int)
data_completa['Price Each']= data_completa['Price Each'].astype(float)
data_completa['Order Date']= pd.to_datetime(data_completa['Order Date'])

In [ ]:
#verificar el cambio de tipo de datos
data_completa.dtypes

### Tarea 3: Analisis 

### ¿Cuál fue el mejor mes en venta?¿ Cuanto se gano ese mes?

In [ ]:
#crear una nueva columna del mes
data_completa['Month Order']=data_completa['Order Date'].dt.month

In [ ]:
#crear una nueva columna de las ventas
data_completa['Sales']= data_completa['Quantity Ordered']*data_completa['Price Each']

In [ ]:
data_completa

In [ ]:
data_completa.groupby(by='Month Order').sum().sort_values(by='Sales',ascending=False)

In [ ]:
meses= range(1,13)
ventas= data_completa.groupby(by='Month Order').sum()['Sales']

In [ ]:
#verlo en grafica
plt.bar(meses,ventas)
plt.xlabel("MESES")
plt.ylabel("VENTAS")
plt.show()

#### el mes en donde mas se vendio fue diciembre

### ¿Que ciudad tuvo mayor ventas?

In [ ]:
#agregar columna ciudad
def obtener_ciudad(city):
    return city.str.split(",", expand=True)[1]
    
def obtener_estado(estado):
    return estado.str.split(",", expand=True)[2]
data_completa['City']=obtener_ciudad(data_completa['Purchase Address'])+' '+obtener_estado(data_completa['Purchase Address'])


In [ ]:
data_completa

In [ ]:
data_completa.groupby(by='City').sum().sort_values(by='Sales',ascending=False)

In [ ]:
ventas_ciudad=data_completa.groupby('City').sum()['Sales']
ciudades=[ciudad for ciudad, df in data_completa.groupby('City')]

In [ ]:
#verlo en grafica
plt.bar(ciudades,ventas_ciudad)
plt.xticks(ciudades,rotation='vertical', size=8)
plt.xlabel("ciudades")
plt.ylabel("ventas")
plt.show()

#### la ciudad de mas ventas es San Francisco CA 94016	

### ¿A qué hora debemos mostrar anuncios para maximizar la probabilidad de que el cliente compre el producto?

In [ ]:
data_completa

In [ ]:
data_completa['Hour Order']=data_completa['Order Date'].dt.hour

In [ ]:
data_completa.groupby('Hour Order').sum().sort_values('Quantity Ordered',ascending==False)

In [ ]:
cantidadVendida_horas= data_completa.groupby('Hour Order').sum()['Quantity Ordered']
horas= [hora for hora, df in data_completa.groupby('Hour Order')]
plt.plot(horas,cantidadVendida_horas)
plt.grid()
plt.show

#### la hora en donde se venden más  por la noche es a las 7 pm y  por la mañana- tarde es de las 10 am a las 12 pm, donde hay mayor probabilidad que si se lanza un anuncio, ocurra la venta

### ¿Qué productos se venden juntos con más frecuencia?

In [ ]:
#duplicated(keep = first obtiene el primero, last obtiene el ultimo , False obtiene todos los registros)
df=data_completa[data_completa['Order ID'].duplicated(keep= False)]

In [ ]:
#df.groupby('Order ID')['Product'] se agrupa por orden y por agrupacion se tiene el nombre de los productos
df['Grouped']= df.groupby('Order ID')['Product'].transform(lambda  x: ",".join(x))

In [ ]:
df

In [ ]:
#eliminar las filas duplicadas ya que por cada fila se hizo la union de los productos
df.drop_duplicates(inplace=True)

In [ ]:
df

In [ ]:
from itertools import combinations
from collections import Counter
count = Counter()
for row in df['Grouped']:
    row_list= row.split(',')
    count.update(Counter(combinations(row_list,2)))
    
for key, value in count.most_common(12):
    print(key, value)

#### los productos que con mayor frecuencia se venden juntos

## ¿Qué producto se vendió más? ¿Por qué crees que vendió más?

In [ ]:
data_completa.groupby('Product').sum().sort_values('Quantity Ordered', ascending= False)['Quantity Ordered']

In [ ]:
#verlo en grafico
productos= [product for product,df in data_completa.groupby('Product')]
ventas_productos= data_completa.groupby('Product').sum()['Quantity Ordered']
plt.bar(productos,ventas_productos)
plt.xticks(productos, rotation='vertical', size=15)
plt.xlabel("productos")
plt.ylabel("ventas")
plt.show()


#### El producto que mayor se vendio fue AAA Batteries (4-pack), ya que es un producto de menor costo

In [ ]:
#comparar con los precios de los productos
precios= data_completa.groupby('Product').mean()['Price Each']

In [ ]:
#poner dos graficos en uno solo
fit,ax1= plt.subplots()
ax2= ax1.twinx()

ax1.bar(productos,ventas_productos)
ax2.plot(productos,precios, color='red')

ax1.set_xlabel("productos")
ax1.set_ylabel("ventas")
ax2.set_ylabel("precios")
ax1.set_xticklabels(productos, rotation='vertical', size=15)
plt.show()